In [ ]:
"""
@Description :   
@Author      :   Xubo Luo 
@Time        :   2024/06/16 20:32:35
"""


In [6]:
import math
from nltk.translate.bleu_score import sentence_bleu

log_avg = (math.log(9 / 9) + math.log(5 / 8) + math.log(2 / 7) + math.log(1 / 6)) / 4
bp = 1  # l_pred == l_true  bp=1
print(math.exp(log_avg))  # 0.41535092372063953

r = sentence_bleu(
    references=[list('这花儿开的真美丽啊'), list('这花开的真美丽啊')],
    hypothesis=list('这开的花儿真美丽啊'),
)
print(r)  # 0.41535092372063953

0.4153509237206396
0.4153509237206396


In [4]:
print([list('这花儿开的真美丽啊')])
print(list('这开的花儿真美丽啊'))

[['这', '花', '儿', '开', '的', '真', '美', '丽', '啊']]
['这', '开', '的', '花', '儿', '真', '美', '丽', '啊']


In [1]:
ch_file = 'tgt.txt'
en_file = 'src.txt'

with open(ch_file, 'r', encoding='utf-8') as f:
    ch_lines = f.readlines()
with open(en_file, 'r', encoding='utf-8') as f:
    en_lines = f.readlines()

with open('ch_en.txt', 'w', encoding='utf-8') as f:
    for ch, en in zip(ch_lines, en_lines):
        if en.strip().endswith(' '):
            en = en.strip()[:-1]
        f.write(en.strip() + ' ' + ch.strip() + '\n')

f.close()